# Interacting with CerebralCortex Data

Cerebral Cortex is MD2K's big data cloud tool designed to support population-scale data analysis, visualization, model development, and intervention design for mobile-sensor data. It provides the ability to do machine learning model development on population scale datasets and provides interoperable interfaces for aggregation of diverse data sources.

This page provides an overview of the core Cerebral Cortex operations to familiarilze you with how to discover and interact with different sources of data that could be contained within the system.

_Note:_ While some of these examples are showing generated data, they are designed to function on real-world mCerebrum data and the signal generators were built to facilitate the testing and evaluation of the Cerebral Cortex platform by those individuals that are unable to see those original datasets or do not wish to collect data before evaluating the system.

## Import packages
Python projects always require a number of imports and the specifics are located in the `util/dependencies.py` file.  The `settings` import specifies the specific `USER_ID` that is utilized within this demonstration.  These _ids_ are the unique user identifiers within Cerebral Cortex.

In [1]:
%reload_ext autoreload
from util.dependencies import *
from cerebralcortex.util.helper_methods import get_study_names

## CerebralCortex Configurations Path

In [2]:
cc_config = "/home/jovyan/cc_conf/"

## List all the available studies

In [3]:
get_study_names(cc_config)

['default']

## Create CerebralCortex object
The __Kernel__ object is the main entry point to the Cerebral Cortex system. It is necessary to pass a configuration directory that tells it all the different parameters it needs to communicate with its other components.  You can examine the details of these configurations for this server by looking at the files contained in the `cc_conf` folder.

In [4]:
CC = Kernel(cc_config, study_name="default")

## Getting help
These are the typical ways to learn more about the code and objects within Cerebral Cortex.
1. Intelligent context help by typing the object or class into a cell followed by the period, `.`, then when you press `<tab>` a popup will appear showing additional information about the object or method. Uncomment the first line to try it out.
2. Formatting the commands with a question mark retrieves the documentation strings and examples when appropriate.  `? CC.list_streams`
3. Reading the documentation on our site: https://cerebralcortex-kernel.readthedocs.io/en/latest/

In [5]:
? CC.list_streams

Signature:  CC.list_streams() -> List[str]
Docstring:
Get all the available stream names with metadata

Returns:
    List[str]: list of available streams metadata

Examples:
    >>> CC = Kernel("/directory/path/of/configs/", study_name="default")
    >>> CC.list_streams()
File:      /opt/conda/lib/python3.8/site-packages/cerebralcortex_kernel-3.3.0-py3.8.egg/cerebralcortex/kernel.py
Type:      method


## Generate some sample data for phone battery
This helper method utilizes Cerebral Cortex (`CC`), the `USER_ID`, and a `stream_name` to generate fake data for for the purposes of these examples.  If you have real-world data, this step can be skipped and your stream names adjusted to make your dataset. This is disabled for this demonstration to not create too much data at once.

In [6]:
gen_phone_battery_data(CC, user_id=USER_ID, stream_name="BATTERY--org.md2k.phonesensor--PHONE")

2020-08-08 05:03:15,495 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f98ec777ca0>: Failed to establish a new connection: [Errno -2] Name or service not known')': /auth/tokens/demo
2020-08-08 05:03:15,498 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f98ec777490>: Failed to establish a new connection: [Errno -2] Name or service not known')': /auth/tokens/demo
2020-08-08 05:03:15,500 WARNING Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f98ec777340>: Failed to establish a new connection: [Errno -2] Name or service not known')': /auth/tokens/demo


## List available streams in CC
One of the first things a researcher typically wants to know is what data is available to explore.  The kernel offers a couple of methods to facilitate this. The first, `list_streams`, is shown below and exposes all the available streams within the system. 

In [7]:
streams = CC.list_streams()
for stream in streams:
    print(stream)

iot-data-stream
gps--org.md2k.clusters
battery--org.md2k.phonesensor--phone
stress-test-stream-name-temporary-remove-it


### Search streams by name
For larger deployments, the list of all streams may be too long to easily sort through, or you may be interested in a specific type of information. In this case, the second method `search_stream` would be more applicable. This search returns streams that have a substring match of the search parameter.

In [8]:
results = CC.search_stream("battery")
for result in results:
    print(result)

battery--org.md2k.phonesensor--phone


## Get stream data
Once a stream is identified by name, it needs to be loaded into a `DataStream` object by calling `get_stream`.  This pulls into a single object all the metadata associated with the stream as well as a reference to the data so that it can be accessed as needed.

In [9]:
battery_data_stream = CC.get_stream("BATTERY--org.md2k.phonesensor--PHONE")

## Print stream statistics
The summary method displays some basic statistics about the datastream such as the number of points as well as max, mean, stdev, and min values.  These statistics are shown for each column of data in the stream.

In [10]:
battery_data_stream.summary()

+-------+------------------+------------------------------------+-------+
|summary|battery_level     |user                                |version|
+-------+------------------+------------------------------------+-------+
|count  |18981             |18981                               |18981  |
|mean   |95.44044044044044 |null                                |1.0    |
|stddev |2.9077665515168256|null                                |0.0    |
|min    |90                |00000000-afb8-476e-9872-6472b4e66b68|1      |
|25%    |93                |null                                |1      |
|50%    |95                |null                                |1      |
|75%    |98                |null                                |1      |
|max    |100               |00000000-afb8-476e-9872-6472b4e66b68|1      |
+-------+------------------+------------------------------------+-------+



## Print stream data
Any datastream can be printed or visualized to the screen; however, it is important to limit, in this case to 3, the number of rows to show.  Streams can contain millions to billions of samples depending on the size of the system and even for the case of a single individual wearing a motion-capture band, this number can exceed 30,000,000 samples for a short two week study.  Cerebral Cortex defaults to settings that try to not load all the data unless needed.

This example prints the first 3 rows of the loaded battery stream and it contains 5 columns. 
- __timestamp__: This is the time in UTC that the sample was recorded at
- __localtime__: This is the time in the local timezone that the sample was recorded at
- __battery_level__: This is the battery percentage of the smartphone device
- __version__: This is the Cerebral Cortex version code assigned to this stream.
- __user__: This is the specific UUID that identifies the user that owns this data point

In [11]:
battery_data_stream.show(3, truncate=False) 

+-------------------+-------------------+-------------+------------------------------------+-------+
|timestamp          |localtime          |battery_level|user                                |version|
+-------------------+-------------------+-------------+------------------------------------+-------+
|2019-01-09 11:47:27|2019-01-09 06:47:27|93           |00000000-afb8-476e-9872-6472b4e66b68|1      |
|2019-01-09 11:47:28|2019-01-09 06:47:28|93           |00000000-afb8-476e-9872-6472b4e66b68|1      |
|2019-01-09 11:47:29|2019-01-09 06:47:29|93           |00000000-afb8-476e-9872-6472b4e66b68|1      |
+-------------------+-------------------+-------------+------------------------------------+-------+
only showing top 3 rows



## Print stream metadata
Each stream contains 

- __name__: The complete string name of this stream
- __description__: A text description of this stream
- __data_descriptor__: A list of objects that describe the data components of the stream (e.g. battery_level)
    - ...
    - __name__: data descriptor name
    - __type__: the object type (e.g. integer, float, string, ...)
    - __optional_fields__: any number of arbitrary fields can be added when creating a stream and will appear here 
    - ...
- __annotations__: Currently unused but designed to link streams together such as a **data quality** and the corresponding **raw** stream
- __input_streams__: Currently unused but designed to specify which streams were utilized to generate this stream
- __modules__: Metadata about the algorithm/code module the generated this data
  - __name__: The name of the code module
  - __version__: The version of the code module
  - __attributes__: Arbitrary attributes specified by _key-value_ pairs
  - __authors__: A set of author names and emails


In [12]:
metadata = battery_data_stream.get_metadata()
print(metadata)

{
    "annotations": [],
    "data_descriptor": [
        {
            "attributes": {
                "description": "current battery charge"
            },
            "name": "level",
            "type": null
        }
    ],
    "description": "mobile phone battery sample data stream.",
    "input_streams": [],
    "modules": [
        {
            "attributes": {
                "attribute_key": "attribute_value"
            },
            "authors": [
                {
                    "nasir ali": "nasir.ali08@gmail.com"
                }
            ],
            "name": "battery",
            "version": "1.2.4"
        }
    ],
    "name": "battery--org.md2k.phonesensor--phone",
    "study_name": "default"
}


## Filter Data

Cerebral Cortex returns all data associated with a stream name, which is great for performing operations and intial exploration; however, it allows for the filtering of these streams of data to isolate certain criterias such as value ranges or specific columns or users.

### Filter data by data column
The first major filtering capability allows for named columns to have logical operations applied to them. The `filter` method is applicable to the data stream object and accepts three parameters.  
- column name: (e.g. battery_level)
- operation: (e.g. >, <, ==, >=, ...)
- criteria: (e.g. 97)



In [13]:
filtered_data = battery_data_stream.filter("battery_level>97")
filtered_data.show(3,truncate=False)

+-------------------+-------------------+-------------+------------------------------------+-------+
|timestamp          |localtime          |battery_level|user                                |version|
+-------------------+-------------------+-------------+------------------------------------+-------+
|2019-01-09 11:39:09|2019-01-09 06:39:09|98           |00000000-afb8-476e-9872-6472b4e66b68|1      |
|2019-01-09 11:39:10|2019-01-09 06:39:10|98           |00000000-afb8-476e-9872-6472b4e66b68|1      |
|2019-01-09 11:39:11|2019-01-09 06:39:11|98           |00000000-afb8-476e-9872-6472b4e66b68|1      |
+-------------------+-------------------+-------------+------------------------------------+-------+
only showing top 3 rows



### Filter data by user
User filtering is a special case due to the way Cerebral Cortex stores data and a dedicated method, `filter_user`, is provided which accepts a single `USER_ID` as input.  This example illustrates filtering by the prior user id.

In [14]:
filtered_user_data = battery_data_stream.filter_user("00000000-afb8-476e-9872-6472b4e66b68")
filtered_user_data.show(3,truncate=False)

+-------------------+-------------------+-------------+------------------------------------+-------+
|timestamp          |localtime          |battery_level|user                                |version|
+-------------------+-------------------+-------------+------------------------------------+-------+
|2019-01-09 11:47:27|2019-01-09 06:47:27|93           |00000000-afb8-476e-9872-6472b4e66b68|1      |
|2019-01-09 11:47:28|2019-01-09 06:47:28|93           |00000000-afb8-476e-9872-6472b4e66b68|1      |
|2019-01-09 11:47:29|2019-01-09 06:47:29|93           |00000000-afb8-476e-9872-6472b4e66b68|1      |
+-------------------+-------------------+-------------+------------------------------------+-------+
only showing top 3 rows



### Filter data by version
Version filtering is a special case due to the way Cerebral Cortex stores data. A dedicated method, `filter_version`, is provided which accepts a single version as input. 

In [15]:
filtered_version_data = battery_data_stream.filter_version(1)
filtered_version_data.show(3,truncate=False)

+-------------------+-------------------+-------------+------------------------------------+-------+
|timestamp          |localtime          |battery_level|user                                |version|
+-------------------+-------------------+-------------+------------------------------------+-------+
|2019-01-09 11:47:27|2019-01-09 06:47:27|93           |00000000-afb8-476e-9872-6472b4e66b68|1      |
|2019-01-09 11:47:28|2019-01-09 06:47:28|93           |00000000-afb8-476e-9872-6472b4e66b68|1      |
|2019-01-09 11:47:29|2019-01-09 06:47:29|93           |00000000-afb8-476e-9872-6472b4e66b68|1      |
+-------------------+-------------------+-------------+------------------------------------+-------+
only showing top 3 rows



## Convert datastream object into Pandas dataframe
The data representations and visualizations that have been shown so far provide a way for basic data inspections; however, these are not directly suitable for more complex interactions or analysis.  Cerebral Cortex provide a `to_pandas` method to transform the datastream data into a [Pandas](https://pandas.pydata.org/) dataframe object. From this point, anything that Pandas can do is supported.


In [16]:
pdf = battery_data_stream.toPandas()
pdf

,timestamp,localtime,battery_level,user,version
0,2019-01-09 11:47:27,2019-01-09 06:47:27,93,00000000-afb8-476e-9872-6472b4e66b68,1
1,2019-01-09 11:47:28,2019-01-09 06:47:28,93,00000000-afb8-476e-9872-6472b4e66b68,1
...,...,...,...,...,...
18979,2019-01-09 11:44:52,2019-01-09 06:44:52,95,00000000-afb8-476e-9872-6472b4e66b68,1
18980,2019-01-09 11:44:53,2019-01-09 06:44:53,94,00000000-afb8-476e-9872-6472b4e66b68,1


## Perform windowing operation on data
Many times it is preferable to group the data into windows before applying an algorithm or computation to the data.  The basic windowing function groups data into non-overlapping chunks and returns a data stream with each cell containing all the data associated with that particular window.

In [17]:
windowed_data = battery_data_stream.window(windowDuration=60)

### Sliding windows
Another common windowing technique can be accomplished by adding an `offset` parameter to the parameter list which causes the windows to move by a partial window size instead of the whole window.

In [18]:
windowed_data = battery_data_stream.window(windowDuration=60, slideDuration=5)

## Compute some basic stats
Cerebral Cortex provides computationally efficient helper functions for generating basic statistics over the datastream. These functions include: _average, sqrt, sum, variance, stdev, min, max_

In [19]:
from cerebralcortex.algorithms.stats.features import statistical_features

stats_features = statistical_features(windowed_data)
stats_features.show(4, False)

/usr/local/spark/python/pyspark/sql/pandas/group_ops.py:73: UserWarning:

It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.



+-------------------+-------------------+------------------------------------+-------+-------------------+-------------------+------------------+--------------------+--------------------+----------------------+-----------------+-----------------+------------------+------------------+-----------------+
|timestamp          |localtime          |user                                |version|start_time         |end_time           |battery_level_mean|battery_level_median|battery_level_stddev|battery_level_variance|battery_level_max|battery_level_min|battery_level_skew|battery_level_kurt|battery_level_sqr|
+-------------------+-------------------+------------------------------------+-------+-------------------+-------------------+------------------+--------------------+--------------------+----------------------+-----------------+-----------------+------------------+------------------+-----------------+
|2019-01-09 11:39:09|2019-01-09 06:39:09|00000000-afb8-476e-9872-6472b4e66b68|1      |2019-